In [1]:
import numpy as np
np.random.seed(5)

from keras.layers import Input, Dense, SimpleRNN
from keras.models import Model
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras import backend as K

In [2]:
# 1. LECTURA DEL SET DE DATOS
# ===========================================================
nombres = open('animales.txt','r').read()
nombres = nombres.lower()

In [3]:
# Crear diccionario (listado de caracteres que no se repiten)
alfabeto = list(set(nombres))
tam_datos, tam_alfabeto = len(nombres), len(alfabeto)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (tam_datos, tam_alfabeto))


En total hay 393 caracteres, y el diccionario tiene un tamaño de 31 caracteres.


In [4]:
# Conversión de caracteres a índices y viceversa
car_a_ind = { car:ind for ind,car in enumerate(sorted(alfabeto))}
ind_a_car = { ind:car for ind,car in enumerate(sorted(alfabeto))}
#print(car_a_ind)
#print(ind_a_car)


In [5]:
ind_a_car

{0: '\n',
 1: ' ',
 2: 'a',
 3: 'b',
 4: 'c',
 5: 'd',
 6: 'e',
 7: 'f',
 8: 'g',
 9: 'h',
 10: 'i',
 11: 'j',
 12: 'k',
 13: 'l',
 14: 'm',
 15: 'n',
 16: 'o',
 17: 'p',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'y',
 24: 'z',
 25: 'á',
 26: 'é',
 27: 'í',
 28: 'ó',
 29: 'ú',
 30: 'ü'}

In [6]:
car_a_ind

{'\n': 0,
 ' ': 1,
 'a': 2,
 'b': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'h': 9,
 'i': 10,
 'j': 11,
 'k': 12,
 'l': 13,
 'm': 14,
 'n': 15,
 'o': 16,
 'p': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'y': 23,
 'z': 24,
 'á': 25,
 'é': 26,
 'í': 27,
 'ó': 28,
 'ú': 29,
 'ü': 30}

In [7]:
# 2. MODELO
# ===========================================================
n_a = 25    # Número de unidades en la capa oculta
entrada  = Input(shape=(None,tam_alfabeto))
a0 = Input(shape=(n_a,))

celda_recurrente = SimpleRNN(n_a, activation='tanh', return_state = True)
capa_salida = Dense(tam_alfabeto, activation='softmax')

salida = []
hs, _ = celda_recurrente(entrada, initial_state=a0)
salida.append(capa_salida(hs))
modelo = Model([entrada,a0],salida)
#modelo.summary()

opt = SGD(lr=0.0005)
modelo.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [9]:
# 3. EJEMPLOS DE ENTRENAMIENTO
# ===========================================================

# Crear lista con ejemplos de entrenamiento y mezclarla aleatoriamente
with open("animales.txt") as f:
    ejemplos = f.readlines()
ejemplos = [x.lower().strip() for x in ejemplos]
np.random.shuffle(ejemplos)

# Crear ejemplos de entrenamiento usando un generador
def train_generator():
    while True:
        # Tomar un ejemplo aleatorio
        ejemplo = ejemplos[np.random.randint(0,len(ejemplos))]

        # Convertir el ejemplo a representación numérica
        X = [None] + [car_a_ind[c] for c in ejemplo]

        # Crear "Y", resultado de desplazar "X" un caracter a la derecha
        Y = X[1:] + [car_a_ind['\n']]

        # Representar "X" y "Y" en formato one-hot
        x = np.zeros((len(X),1,tam_alfabeto))
        onehot = to_categorical(X[1:],tam_alfabeto).reshape(len(X)-1,1,tam_alfabeto)
        x[1:,:,:] = onehot
        y = to_categorical(Y,tam_alfabeto).reshape(len(X),tam_alfabeto)

        # Activación inicial (matriz de ceros)
        a = np.zeros((len(X), n_a))

        yield [x, a], y

In [10]:
# 4. ENTRENAMIENTO
# ===========================================================
BATCH_SIZE = 80			# Número de ejemplos de entrenamiento a usar en cada iteración
NITS = 1000			# Número de iteraciones

for j in range(NITS):
    historia = modelo.fit_generator(train_generator(), steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0)

    # Imprimir evolución del entrenamiento cada 1000 iteraciones
    if j%10 == 0:
        print('\nIteración: %d, Error: %f' % (j, historia.history['loss'][0]) + '\n')
    
   # if j%1000 == 0:
    #    print('\nIteración: %d, Error: %f' % (j, historia.history['loss'][0]) + '\n')


<ipython-input-10-2a610e8d4696>:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historia = modelo.fit_generator(train_generator(), steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0)



Iteración: 0, Error: 3.454515


Iteración: 10, Error: 3.416944


Iteración: 20, Error: 3.379691


Iteración: 30, Error: 3.360268


Iteración: 40, Error: 3.327848


Iteración: 50, Error: 3.300964


Iteración: 60, Error: 3.266415


Iteración: 70, Error: 3.243536


Iteración: 80, Error: 3.218822


Iteración: 90, Error: 3.172513


Iteración: 100, Error: 3.166683


Iteración: 110, Error: 3.126048


Iteración: 120, Error: 3.116546


Iteración: 130, Error: 3.091487


Iteración: 140, Error: 3.061236


Iteración: 150, Error: 3.085630


Iteración: 160, Error: 3.025598


Iteración: 170, Error: 3.009012


Iteración: 180, Error: 2.997040


Iteración: 190, Error: 2.977726


Iteración: 200, Error: 2.994984


Iteración: 210, Error: 2.942580


Iteración: 220, Error: 2.959760


Iteración: 230, Error: 2.921436


Iteración: 240, Error: 2.909495


Iteración: 250, Error: 2.954332


Iteración: 260, Error: 2.916959


Iteración: 270, Error: 2.860010


Iteración: 280, Error: 2.923119


Iteración: 290, Error: 2

In [11]:
# 5. GENERACIÓN DE NOMBRES USANDO EL MODELO ENTRENADO
# ===========================================================
def generar_nombre(modelo,car_a_num,tam_alfabeto,n_a):
    # Inicializar x y a con ceros
    x = np.zeros((1,1,tam_alfabeto,))
    a = np.zeros((1, n_a))

    # Nombre generado y caracter de fin de linea
    nombre_generado = ''
    fin_linea = '\n'
    car = -1

    # Iterar sobre el modelo y generar predicción hasta tanto no se alcance
    # "fin_linea" o el nombre generado llegue a los 50 caracteres
    contador = 0
    while (car != fin_linea and contador != 50):
          # Generar predicción usando la celda RNN
          a, _ = celda_recurrente(K.constant(x), initial_state=K.constant(a))
          y = capa_salida(a)
          prediccion = K.eval(y)

          # Escoger aleatoriamente un elemento de la predicción (el elemento con
          # con probabilidad más alta tendrá más opciones de ser seleccionado)
          ix = np.random.choice(list(range(tam_alfabeto)),p=prediccion.ravel())

          # Convertir el elemento seleccionado a caracter y añadirlo al nombre generado
          car = ind_a_car[ix]
          nombre_generado += car

          # Crear x_(t+1) = y_t, y a_t = a_(t-1)
          x = to_categorical(ix,tam_alfabeto).reshape(1,1,tam_alfabeto)
          a = K.eval(a)

          # Actualizar contador y continuar
          contador += 1

          # Agregar fin de línea al nombre generado en caso de tener más de 50 caracteres
          if (contador == 50):
            nombre_generado += '\n'

    print(nombre_generado)

In [12]:

# Generar 100 ejemplos de nombres generados por el modelo ya entrenado
for i in range(1000):
    generar_nombre(modelo,car_a_ind,tam_alfabeto,n_a)

a

e

a

armalospeamesalafna

e

epúapdptaéka

atnreralatsigcoo

ritanela



ltraegasoslomaplco



a

o

aa

a



íraphauáe

ísroisltonalaltosanaeébtpeáuplpoúasotmptcapanopa



alo

iüci

aóma

esaae

ageíélcaiteo

ánteraatmamilerapgíúagpaelofraharpasacnlpaorsalüia

tran

npbt

a

atmanaülolaak

amta

rianmapra

nttogppepíaamgeroteumatpalepdot

tba

latda

ale

lmpepeirbantla

clo

repanentiireirana

a

zoralo

gpa

oneeitrauakaauco

ce

n

oo

ilipslijtpo

tpigpropaisabaleühranerppa

gino

üirrafeileapo

lo

 dlpfimopsoilptbp igódio

ezeirpetmaro

ilf

fi

lle

amaóüa



hbalnllemaetróo

n

uídasaatiloe

no

ant

emevpeilort

bjopotrnlarpottpinmalpita

otra

lavmaroseinpa

üfentabon

lttdialolgncdmlarpgaatmeba

ieflleapo

ec

pto

isnalalokcéntebortspaarpa

ptampe

famzipia

lopaiácfacersaceisaho

rpopapeicerdalaú

rfopacáginlagezaa

aprzji ope



kigaraantelcaraticso

lolsieblo

núáéma

horaíiatnpti

llnoiiir

onltfpeja



islapao

ra

lp

anrloanma

o

llaipag voraaa

slatpiroro

uu